In [2]:
import sys
sys.path.append("..")

from src.uilc import *

# Theories and Related routines

The major goal of LEDs array design for target plane intensity uniformity is minimizing derivation of intensity with finite number of LEDs, by determining number of LEDs, minimizing LEDs array area, and maximzing average power of plane intensity. 

There are two major approaches in location determination, analytic methods and metaheuristic methods. This project implemented one of analytical method. improvement of method, and addtionnal heuristic methods based on fore two methods. Refered methods are described below texts.


## Optical Model of LED

The imperpect Lambertian model is commonly used for hemisphere lenz LED device and far field irradiation. With inverse-square law, optical model of LED can be represented with target-LED distance, $r$, viewing angle, $\theta$, Lambertian coefficient, $s$, and $1m$ irradiation intensity $I_0$.

$$I(r, \theta) = \frac{I_0}{r^2} \cos^s (\theta)$$

## Initial Conditions of System

* $s$: Optical model parameter: Lambertian coefficient.
* $H$: Distance  between target plane and LED plane.
* $W$ : Width of permitted area(Linear). 
* $W_1, W_2$: Widths of permitted area(Rectangular).


## Methods

### 1. Esc method

> Moreno, I., Avendaño-Alejo, M., & Tzonchev, R. I. (2006). Designing light-emitting diode arrays for uniform near-field irradiance. Applied optics, 45(10), 2265-2272.

This method is using expanded Sparrow's Criterion[^1]

<table style="border-radius:8px;width:100%;">
<th style="text-align:center;background-color:rgb(0, 0, 0); color:white; border-top-left-radius: 10px;width:20%;">
Def</th>
<th style="text-align:left;">
Sparrow Criterion </th>
<tr style="text-align:center;">
<td colspan="2">
For two LEDs and superposition of those intensity function,

$$I(t) = I(-d/2,t) + I(d/2,t)$$

The maximum distance $d$ is yielded $\frac{\partial^2 I}{\partial t^2}|_{t=0} = 0$.


</td>
</tr>
<tr><td ></td> <td > Expended Sparrow Criterion</td></tr>
<tr style="text-align:center;">
    <td colspan="2">
    
For $N$ number of LEDs which are uniformly distributed with distance $d$ each other, the flat(or uniform) condition of intensity is 

$$\frac{\partial^2 I_N}{\partial t^2}|_{t=0} = 0$$

where, 

$$I_N (t) = \sum_{i=1}^N I( (- \frac{N}{2} + (i-1)) d, t)$$


</td>
</tr>
</table>


LEDs are uniformly distributed, their uniform distance $r$ is calculated from coefficient $d$ and distance between target plane and LED plane $h$.

$$r = d \cdot h$$

Finding root of next equations for linear and rectangular array yield flat coefficient $d$.

linear

$$f(d) = \sum_{n=1}^N \frac{[1-(m+3)(N+1-2n)^2 (\frac{d}{2})^2]}{[(N+1-2n)^2 (\frac{d}{2})^2]^{(m+6)/2}}$$

rectangular

$$f(d) = \sum_{i=1}^N \sum_{j=1}^M  \frac{\{1-[(m+3)(N+1-2i)^2 -(M+1-2j)^2](\frac{d}{2})^2\})}{(\{ [N+1-2i)^2 + (M+1-2j)^2](\frac{d}{2})^2+1\}^{(m+6)/2}}$$

[^1]: R. Barakat, “Application of apodization to increase two-point resolution by the sparrow criterion. i. coherent248 illumination,” J. Opt. Soc. Am. 52, 276–283 (1962).

#### Validation of implementated routines

##### Linear Array

Flat conditionn $d$ for two LEDs,

$$d = \sqrt{\frac{4}{s+3}}$$

$s = 80.7$ then, $d = 0.219$.


In [9]:
esc.coefficient(80.7, 2, approx=True)

0.2186086119279843

For  $N =7$ and $ s = 80.7$,

$$d = 0.135$$

In [3]:
esc.coefficient(80.7, 7)

0.13478354980171428

For $N =4$ and $s = 64.66$,

$$d = 0.189$$

In [4]:
esc.coefficient(64.66, 4)

0.1890009186203515

**Approximation**

For $N > 4$ and $s > 30$

> **Note** I think this condition has errors. Mostly, it fits well when $N< 4, s<30$.

$$d = \sqrt{\frac{3.2773}{s+4.2539}}$$

In [5]:
s = 30.2
N = 9
print(esc.coefficient(s, N, approx=True))
print(esc.coefficient(s,N, approx=False))
s = 30
N = 10
print(esc.coefficient(s, N, approx=True))
print(esc.coefficient(s,N, approx=False))

0.3084174233435047
0.19358628934893632
0.18654998378436424
0.18654998378436424


##### Square Array

For $2 \times 2$ LEDs,

$$d = \sqrt{\frac{4}{s+2}}$$

In [6]:
print(esc.coefficient(50, N=2, M=2, shape="R"))
print(math.sqrt(4/(50+2)))

0.2773500981126146
0.2773500981126146


For $N = M = 7$ and $s= 50$

$$d= 0.17$$

In [7]:
esc.coefficient(50, N=7, M=7, shape="R")

0.17201102131586457

**Approximation**

For larger than $4 \times 4$ LEDs $N \times M > 4 \times  4$ and $s > 30$

$$d = \sqrt{\frac{1.2125}{s-3.349}}$$

In [8]:
print(esc.coefficient(50, N=7, M=7, shape="R", approx=True))
print(math.sqrt(1.2125/(50-3.349)))

0.16121683647482485
0.16121683647482485


### 2. BC Matching

Boundary Center Matching expansion method.

$$I(\theta, r) = I(H, s;x,t) = I_0 \frac{H^m}{(H^2 + (x-t)^2)^(\frac{s}{2}+1)}$$

The difference function $Di(x)$ of two LEDs, located in center symmetry, intensity contribution between two points, center and boundary is defined as

$$Di(x) : = (I(x, \frac{W}{2} + I(-x, \frac{W}{2})) - I(x,0) + I(-x,0))$$

It can be nomailized to $D(\alpha, d)$ with next equation.

$$D(\frac{W}{H}; \frac{x}{H}) = \frac{H^2}{I_0}Di(x)$$

$x_e$ is a point that makes boundary and center contribution of LEDs same.

$x_m$ is a point for center corresponding to $x=W/2$ for boundary. 

$$x_e = \text{root}(D(d))*H$$
$$x_m = \text{root}(D(d)+D(\alpha/2))*H$$

$\mathbf{P} := [0,x_m), \mathbf{Q} := [x_m,x_e], \mathbf{R} := (x_e, W/2]$

For $x_q \in \mathbf{Q}$ and $x_r \in \mathbf{R}$, there exist $x'_r \in \mathbf{R}$ and $x'_q \in \mathbf{Q}$ such that,

$$I_c(x_q) + I_c(x'_r) = I_b(x_q) + I_c(x'_r)$$
$$I_c(x_r) + I_c(x'_q) = I_b(x_r) + I_c(x'_q)$$

Then, for Esc array which consist of LEDs in $\mathbf{Q}$ region in given area $W$, the LEDs of which the center and boundary intensity contributions are same can be found.

$$\{\overbrace{x_1, x_2, ... , x_{m-1} }^\mathbf{Q}, \underbrace{x_m ,...,x_{n-1} x_n}_\mathbf{R}\}$$


$$\sum_{i=1}^n I_c (x_i) = \sum_{i=1}^n I_b (x_i) $$

Practical approximation are next equations.

$$x_m = (\sqrt{2^{\frac{2}{s+2}}-1}) \cdot H$$
$$x_e = (\frac{1}{6}d_m + \frac{1}{4}\alpha) \cdot H$$

These are assumed as initial guessing in finding algorithm. 

### 3. Integral Equation solve(Direct Solve)

Using a Dirac delta function the location of LEDs can be expressed as distribution function $\sigma(x,y)$

$$\sigma(x,y) = \sum_{i=1}^{N \times M} \delta(x-x_i, y-y_i) $$

Since, it is a distribution function, next property is established.

$$\sigma(x,y) \geq 0$$

$$N \times M = \int_{-\frac{W_y}{2}}^{\frac{W_y}{2}} \int_{-\frac{W_x}{2}}^{\frac{W_x}{2}} \sigma(x,y) \, dx dy$$


Restricting dimension to $1-$ dim, and using intensity function $K(x,t) := I(x,t)$ as kernel function. Target plane intensity $I_{tar}(t)$ is calculated with integral equation.

$$I_{tar}(t) = \int_{-\frac{W}{2}}^{\frac{W}{2}} K(x,t) \sigma(x) \, dx$$

It is a Fredholem integral equation of first kind. There are several appeoaches to solve integral equation but this kernel is a case that those approaches can solve at least in my knowledge.

Practical method is discretizing system and solve linear system.

For $n$ sample points and $\Delta = \frac{W}{n}$,

$$\mathbf{K}_{ij} := K(\Delta i, \Delta j)$$


$$\mathbf{K} \vec{\sigma} = \lambda \vec{\mathcal{1}}$$

$\mathbf{K}$ is a bisymmetric matrix and by M.Kaykobad Theory, it is invertible and has a positive solution with next conditions.

$$1> \sum_{i=1, i \neq j}^{n} \frac{\mathbf{K}_{ij}}{\mathbf{K}_{ii}}$$

$max(n)$ satisfies above condition can be approximated with $n_{app}$.

For $\Delta_n = \frac{W}{Hn}$, $K_N(H;x) = \frac{K(x,0)}{K(0,0)}$

$$2\int_{\frac{W}{2Hn}}^{\frac{W}{2H}} K_N(x) \, dx\geq \sum_{i=1, i\neq j}^n K_N(|i-j|\Delta_n) \Delta_n$$

$$n_{app} = \lfloor \frac{1+ 2 {}_2F_1(1/2, \frac{s+2}{2}, 3/2; - (\frac{W}{2Hn_{app}})^2)}{{}_2F_1(1/2, \frac{s+2}{2}, 3/2; - (\frac{W}{2H})^2)} \rfloor$$

As $n$ increases, Hypergeometric fuction will converge near $max(n)$.



$$n_{app} = \lfloor \frac{3}{{}_2F_1(1/2, \frac{s+2}{2}, 3/2; - (\frac{W}{2H})^2)} \rfloor$$


Real therhold value $max(n)$  is near $n_{app}$ and M.Kaykobad's Thm condition is not describing threshold level condition of postive to negative solution. Real $max(n)$ is usually bigger than M.Kaykobad's condition $n$. 

Solution of system does not satisfy second distribution condtion because the $\lambda$ value is omitted in calculation. Therefore, normalization process is needed. The normalization factor $\lambda_N$ is using approximation value $n_{LED}$. Practically, the bc method solution LED number is used for $n_{LED}$ value.

$$\lambda_{N} = \frac{n_{LED} \cdot n_{max}}{W \sum_{i=1}^{n_{max}} \sigma_i}$$

$$\mathbf{\sigma}_{sol} =  \lambda_{N} \mathbf{\sigma}$$

Final array finding is decribed in Binarization of distribution vector 


### 4. Integral Equation solve(NNLS)

It is using Non-Negative least square method for finding distrubtion vector. It can use increase number of samples for precise result.

In NNLS result, the number of LED $n_{LED}$ can be assumed directly from solution its number of extrem values $+2$ is the $n_{LED}.

## Binarization of distribution vector 

The Dirac-Delta function can be approximated with normal distribution function.

$$\delta(x) \approx \frac{1}{h\sqrt{2 \pi}} \exp(- \frac{x^2}{2 h^2})$$

$$N (h, x_0 ; x) := \frac{1}{h \sqrt{2 \pi}} \exp(- \frac{(x-x_0)^2}{2 h^2})$$

$h$ is standard derivation of distribution but also it is a bandwidth of normal kernel. In kernel density estimation, bandwidth is a free parameter for estimating unkown density function. 

In this calculation this bandwidth depends on discretizing sample number $n_{sam}$. The geometrical space is divided $n_{sam}$ so that bandwidth $h$ is defined

$$h = \frac{W}{2 n_{sam} \cdot k}$$

Emprical value of $k$ is 

$$k= \sqrt{2} \cdot 0.9$$

$$\sigma(x) = \sum_{i=1}^{N \times M} \delta(x-x_i) \approx \sum_{i=1}^{N \times M} N (h, x_i ; x) $$

With direct solve and nnls solution vector $\mathbf{\sigma}_{sol}$ are sample points for optimization of $\{x_i \}_{i=1}^{N \times M}$ for $\sum_{i=1}^{N \times M} N (h, x_i ; x)$. This calculation uses least square method.



### Brief Introduction of Least square method in Scipy

**Least Square Method** is a standard approach in regression analysis to approximate the solution with minimizing the sum of the squares of the residuals. Residual is the difference between an observed value and the fitted value. 

For fitting function $f$ with parameter $\pmb{\beta} = \beta_1, \beta_2, \dots, \beta_n$ and the observed data $[(x_1, y_1), (x_2, y_2), (x_3, y_3), (x_4, y_4), \dots, (x_m, y_m)]$, the residual of $i$-th is defined as 

$$r_i = y_i - f(x_i, \pmb{\beta})$$

Object of least square method is minimizing the sum of squared residuals $S$

$$S = \sum_{i=1}^m r_i^2$$

Scipy library provides various types of function to solve optimizing problems.
All of them are user friendly api of next function `least_squares`. It is a primitive implementation of 
least square algorithm.

`scipy.optimize.least_squares`

|Arguments|Description|
|:--|:--|
|`func`|Callable function that computes residuals with the signature `func(x, *args, **kwargs)`. `x` passed to this function is an ndarray of shpae `(n,)`. It must return a 1-D array-like of shape `(m,)` or a scalar.  That is, `func( x = `$\pmb{\beta}$,  `args =(xdata, ydata)` `)` $= \pmb{r} = \pmb{y} - f(\pmb{x}, \pmb{\beta})$|
|`x0`|Initial guess on independent variables.|
|`args, kwargs`|tuple and dictionary of additional arguments passed to `func(x, *args, **kwargs)`|

There are various optional arguments in `least_squares` function. The above table is just a brief description of used arguments in below codes

# Further Reading

## Metaheuristic methods

This project did not cover metaheuristic methods but those are worth to see.

The examples of metaheuristic methods are

* Stimulated Annealing 
* Genetic Algorithm
* Local Search
* Particle Swarm Optimization.

See details of above methods in next papers

* Z. Su, D. Xue, and Z. Ji, “Designing led array for uniform illumination distribution by simulated annealing algorithm,”257 Opt. Express 20, A843–A855 (2012).
* S. Pal, “Optimization of led array for uniform illumination over a target plane by evolutionary programming,” Appl.259 Opt. 54, 8221–8227 (2015).
* P. Lei, Q. Wag, and H. Zou, “Designing led array for uniform illumination based on local search algorithm,” JEOS:RP2619 (2014).
* . Yu, X. Wei, O. Zhang, and X. Zhang, “Research on illumination uniformity of high-power led array light source,”263 in AIP Conference Proceedings, vol. 1971 (AIP Publishing LLC, 2018), p. 050019.